In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os
import json
import base64
import asyncio
import platform
import requests
import operator
import playwright
import numpy as np
import pandas as pd
import datetime as dt

from enum import Enum
from typing import List
from typing import Dict
from typing import Tuple
from typing import Union
from typing import Literal
from typing import Optional
from typing import Sequence
from typing import Annotated
from typing import TypedDict
from operator import itemgetter

from IPython import display
from IPython.display import HTML
from IPython.display import Image

from langgraph.graph import END
from langgraph.graph import StateGraph
from langgraph.graph import MessageGraph
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.messages import BaseMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.chat import ChatMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.prompts.image import ImagePromptTemplate

from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.runnables.graph import CurveStyle
from langchain_core.runnables.graph import NodeColors
from langchain_core.runnables.graph import MermaidDrawMethod

from langchain import hub
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.agents import create_openai_functions_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_fireworks.chat_models import ChatFireworks

In [3]:

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'Reflection'

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'You are an essay assistant tasked with writing excellent 5-paragraph essays. '
            'Generate the best essay possible for the user\'s request.'
        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

llm = ChatFireworks(
    model='accounts/fireworks/models/mixtral-8x7b-instruct',
    max_tokens=32768
)

generate = prompt | llm

In [5]:
essay = ''
request = HumanMessage(content='Write an essay on why the little prince is relevant in modern childhood')

for chunk in generate.stream({'messages': [request]}):
    print(chunk.content, end='')
    essay += chunk.content

Title: The Relevance of The Little Prince in Modern Childhood

The Little Prince, a novella by Antoine de Saint-Exupéry, has been a beloved classic for generations. Despite being written in the early 20th century, the story remains highly relevant in modern childhood. The tale of the Little Prince's journey through various planets, meeting different characters, offers timeless life lessons that resonate with children today.

Firstly, The Little Prince embodies the spirit of exploration and adventure, which is a significant aspect of modern childhood. With the advent of technology, children are exposed to a world of information and possibilities at their fingertips. The Little Prince's journey, both literal and metaphorical, mirrors this digital age exploration. The story encourages children to venture beyond their comfort zones, to ask questions, and to seek answers, fostering a sense of curiosity and resilience that is crucial in the modern world.

Secondly, The Little Prince explores

### Reflect

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'You are a teacher grading an essay submission. Generate critique and recommendations for the user\'s submission. '
            'Provide detailed recommendations, including requests for length, depth, style, etc.'
        ),
        MessagesPlaceholder(variable_name='messages')
    ]
)

reflect = reflection_prompt | llm

In [7]:
reflection = ''
for chunk in reflect.stream({'messages': [request, HumanMessage(content=essay)]}):
    print(chunk.content, end='')
    reflection += chunk.content

Title: The Relevance of The Little Prince in Modern Childhood

Grade: B+

Your essay provides a clear and well-structured argument for why The Little Prince remains relevant in modern childhood. You have identified several key themes in the novella that resonate with contemporary children, including exploration, human emotions, individuality, and friendship. Your use of specific examples from the text to support your points is commendable.

However, there are a few areas where your essay could be improved:

1. Length: Your essay is well-written, but it is relatively brief. I would recommend expanding upon your ideas to create a more in-depth analysis. For example, you could delve deeper into the specific life lessons that the Little Prince learns on his journey and how they apply to modern childhood.
2. Depth: While you have identified several key themes in the novella, you could further explore the nuances of these themes. For instance, you could discuss how the Little Prince's journe

### Repeat
That's all that's there to it. You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [8]:
for chunk in generate.stream({'messages': [request, AIMessage(content=essay), HumanMessage(content=reflection)]}):
    print(chunk.content, end='')

Title: The Relevance of The Little Prince in Modern Childhood

Grade: A-

The Little Prince, a novella by Antoine de Saint-Exupéry, has maintained its relevance in modern childhood, despite being written in the early 20th century. This essay will explore the significance of exploration, human emotions, individuality, and friendship in The Little Prince and how these themes resonate with contemporary children.

The Little Prince's journey through various planets serves as a metaphor for the different stages of life and the challenges that come with each stage. The story encourages children to be curious, resilient, and confident, preparing them for the challenges and opportunities of the modern world. For instance, the Little Prince's encounter with the king on the first planet teaches children about the absurdity of authority figures who demand obedience without reason. This lesson is particularly relevant in modern childhood, where children are often subjected to rigid hierarchies and

In [9]:
async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({'messages': state})

async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    cls_map = {'ai': HumanMessage, 'human': AIMessage}

    # first message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [cls_map[msg.type](content=msg.content) for msg in messages[1:]]
    res = await reflect.ainvoke({'messages': translated})

    # we treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node('generate', generation_node)
builder.add_node('reflect', reflection_node)
builder.set_entry_point('generate')

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # end after 3 iterations
        return END
    return 'reflect'

builder.add_conditional_edges('generate', should_continue)
builder.add_edge('reflect', 'generate')
graph = builder.compile()

In [10]:
async for event in graph.astream(
    [
        HumanMessage(content='Generate an essay on the topicality of The Little Prince and its message in modern life')
    ],
):
    print(event)
    print('----')

{'generate': AIMessage(content='Title: The Timeless Message of The Little Prince in Modern Life\n\nIntroduction:\nAntoine de Saint-Exupéry\'s "The Little Prince" is a classic novella that has captured the hearts and minds of readers for over seven decades. Its enduring popularity can be attributed to its timeless themes and messages, which remain relevant in modern life. This essay will explore the topicality of The Little Prince and its message in today\'s world.\n\nBody Paragraph 1: The Little Prince as a Reflection of Modern Existentialism\nThe Little Prince is often regarded as a philosophical tale that explores the complexities of human existence. The protagonist\'s journey through various planets, each inhabited by peculiar characters, mirrors the existentialist notion of the absurdity of life. In the modern world, where individuals often grapple with questions of purpose and meaning, The Little Prince serves as a reminder of the importance of introspection and self-discovery.\n\

## Conclusion
Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.